In [0]:
import argparse
import os
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.nn import Parameter
from torch.nn.modules.module import Module
import torch.nn.functional as F
from torchvision import datasets, transforms
import math

import numpy as np
import torch
from torch.nn import Parameter
from torch.nn.modules.module import Module
import torch.nn.functional as F
import torch.nn as nn
import torch.nn.functional as F

In [0]:

class PruningModule(Module):
    def prune_by_percentile(self, q=5.0, **kwargs):
        """
        Note:
             The pruning percentile is based on all layer's parameters concatenated
        Args:
            q (float): percentile in float
            **kwargs: may contain `cuda`
        """
        # Calculate percentile value
        alive_parameters = []
        for name, p in self.named_parameters():
            # We do not prune bias term
            if 'bias' in name or 'mask' in name:
                continue
            tensor = p.data.cpu().numpy()
            alive = tensor[np.nonzero(tensor)] # flattened array of nonzero values
            alive_parameters.append(alive)

        all_alives = np.concatenate(alive_parameters)
        percentile_value = np.percentile(abs(all_alives), q)
        print(f'Pruning with threshold : {percentile_value}')

        # Prune the weights and mask
        # Note that module here is the layer
        # ex) fc1, fc2, fc3
        for name, module in self.named_modules():
            if name in ['fc1', 'fc2', 'fc3']:
                module.prune(threshold=percentile_value)

    def prune_by_std(self, s=0.25):
        """
        Note that `s` is a quality parameter / sensitivity value according to the paper.
        According to Song Han's previous paper (Learning both Weights and Connections for Efficient Neural Networks),
        'The pruning threshold is chosen as a quality parameter multiplied by the standard deviation of a layer’s weights'

        I tried multiple values and empirically, 0.25 matches the paper's compression rate and number of parameters.
        Note : In the paper, the authors used different sensitivity values for different layers.
        """
        for name, module in self.named_modules():
            if name in ['fc1', 'fc2', 'fc3']:
                threshold = np.std(module.weight.data.cpu().numpy()) * s
                print(f'Pruning with threshold : {threshold} for layer {name}')
                module.prune(threshold)


class MaskedLinear(Module):
    r"""Applies a masked linear transformation to the incoming data: :math:`y = (A * M)x + b`

    Args:
        in_features: size of each input sample
        out_features: size of each output sample
        bias: If set to False, the layer will not learn an additive bias.
            Default: ``True``

    Shape:
        - Input: :math:`(N, *, in\_features)` where `*` means any number of
          additional dimensions
        - Output: :math:`(N, *, out\_features)` where all but the last dimension
          are the same shape as the input.

    Attributes:
        weight: the learnable weights of the module of shape
            (out_features x in_features)
        bias:   the learnable bias of the module of shape (out_features)
        mask: the unlearnable mask for the weight.
            It has the same shape as weight (out_features x in_features)

    """
    def __init__(self, in_features, out_features, bias=True):
        super(MaskedLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        # Initialize the mask with 1
        self.mask = Parameter(torch.ones([out_features, in_features]), requires_grad=False)
        if bias:
            self.bias = Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input):
        return F.linear(input, self.weight * self.mask, self.bias)

    def __repr__(self):
        return self.__class__.__name__ + '(' \
            + 'in_features=' + str(self.in_features) \
            + ', out_features=' + str(self.out_features) \
            + ', bias=' + str(self.bias is not None) + ')'

    def prune(self, threshold):
        weight_dev = self.weight.device
        mask_dev = self.mask.device
        # Convert Tensors to numpy and calculate
        tensor = self.weight.data.cpu().numpy()
        mask = self.mask.data.cpu().numpy()
        new_mask = np.where(abs(tensor) < threshold, 0, mask)
        # Apply new weight and mask
        self.weight.data = torch.from_numpy(tensor * new_mask).to(weight_dev)
        self.mask.data = torch.from_numpy(new_mask).to(mask_dev)




class LeNet(PruningModule):
    def __init__(self, mask=False):
        super(LeNet, self).__init__()
        linear = MaskedLinear if mask else nn.Linear
        self.fc1 = linear(784, 512)
        self.fc2 = linear(512, 512)
        self.fc3 = linear(512, 512)
        self.fc4 = linear(512, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        return x


In [0]:
from torch.autograd import Variable
import os
import numpy as np
import math
import scipy
import sys


def softmax_by_row(logits, T = 1.0):
    mx = np.max(logits, axis=-1, keepdims=True)
    exp = np.exp((logits - mx)/T)
    denominator = np.sum(exp, axis=-1, keepdims=True)
    return exp/denominator

def classifier_performance(model, train_loader, test_loader):

    output_train_benign = []
    train_label = []
    for num, data in enumerate(train_loader):
        images,labels = data
        image_tensor= images.to(device)
        img_variable = Variable(image_tensor, requires_grad=True)
        output = model.forward(img_variable)

        train_label.append(labels.numpy())
        output_train_benign.append(softmax_by_row(output.data.cpu().numpy(),T = 1))


    train_label = np.concatenate(train_label)
    output_train_benign=np.concatenate(output_train_benign)

    test_label = []
    output_test_benign = []

    for num, data in enumerate(test_loader):
        images,labels = data

        image_tensor= images.to(device)
        img_variable = Variable(image_tensor, requires_grad=True)

        output = model.forward(img_variable)

        test_label.append(labels.numpy())
        output_test_benign.append(softmax_by_row(output.data.cpu().numpy(),T = 1))


    test_label = np.concatenate(test_label)
    output_test_benign=np.concatenate(output_test_benign)


    train_acc1 = np.sum(np.argmax(output_train_benign,axis=1) == train_label.flatten())/len(train_label)
    test_acc1 = np.sum(np.argmax(output_test_benign,axis=1) == test_label.flatten())/len(test_label)

    print('Accuracy: ', (train_acc1, test_acc1))

    return output_train_benign, output_test_benign, train_label, test_label




def inference_via_confidence(confidence_mtx1, confidence_mtx2, label_vec1, label_vec2):

    #----------------First step: obtain confidence lists for both training dataset and test dataset--------------
    confidence1 = []
    confidence2 = []
    acc1 = 0
    acc2 = 0
    for num in range(confidence_mtx1.shape[0]):
        confidence1.append(confidence_mtx1[num,label_vec1[num]])
        if np.argmax(confidence_mtx1[num,:]) == label_vec1[num]:
            acc1 += 1

    for num in range(confidence_mtx2.shape[0]):
        confidence2.append(confidence_mtx2[num,label_vec2[num]])
        if np.argmax(confidence_mtx2[num,:]) == label_vec2[num]:
            acc2 += 1
    confidence1 = np.array(confidence1)
    confidence2 = np.array(confidence2)

    print('model accuracy for training and test-', (acc1/confidence_mtx1.shape[0], acc2/confidence_mtx2.shape[0]) )


    #sort_confidence = np.sort(confidence1)
    sort_confidence = np.sort(np.concatenate((confidence1, confidence2)))
    max_accuracy = 0.5
    best_precision = 0.5
    best_recall = 0.5
    for num in range(len(sort_confidence)):
        delta = sort_confidence[num]
        ratio1 = np.sum(confidence1>=delta)/confidence_mtx1.shape[0]
        ratio2 = np.sum(confidence2>=delta)/confidence_mtx2.shape[0]
        accuracy_now = 0.5*(ratio1+1-ratio2)
        if accuracy_now > max_accuracy:
            max_accuracy = accuracy_now
            best_precision = ratio1/(ratio1+ratio2)
            best_recall = ratio1
    print('membership inference accuracy is:', max_accuracy)
    return max_accuracy


In [0]:
def print_model_parameters(model, with_values=False):
    print(f"{'Param name':20} {'Shape':30} {'Type':15}")
    print('-'*70)
    for name, param in model.named_parameters():
        print(f'{name:20} {str(param.shape):30} {str(param.dtype):15}')
        if with_values:
            print(param)


def print_nonzeros(model):
    nonzero = total = 0
    for name, p in model.named_parameters():
        if 'mask' in name:
            continue
        tensor = p.data.cpu().numpy()
        nz_count = np.count_nonzero(tensor)
        total_params = np.prod(tensor.shape)
        nonzero += nz_count
        total += total_params
        print(f'{name:20} | nonzeros = {nz_count:7} / {total_params:7} ({100 * nz_count / total_params:6.2f}%) | total_pruned = {total_params - nz_count :7} | shape = {tensor.shape}')
    print(f'alive: {nonzero}, pruned : {total - nonzero}, total: {total}, Compression rate : {total/nonzero:10.2f}x  ({100 * (total-nonzero) / total:6.2f}% pruned)')




In [5]:
batch_size=64
test_batch_size=1000
epochs=50
lr=1e-3
no_cuda=False
seed=1337
log_interval=100
logfile='log.txt'
sensitivity=2  #sensitivity value that is multiplied to layer's std in order to get threshold value

# Control Seed
torch.manual_seed(seed)

# Select Device
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else 'cpu')
if use_cuda:
    print("Using CUDA!")
    torch.cuda.manual_seed(seed)
else:
    print('Not using CUDA!!!')

Using CUDA!


In [6]:
train_loader = torch.utils.data.DataLoader(datasets.FashionMNIST('data', train=True, download=True,transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])),batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(datasets.FashionMNIST('data', train=False, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])),batch_size=test_batch_size, shuffle=False)


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw
Processing...
Done!


In [7]:
model = LeNet(mask=True).to(device)

print(model)
print_model_parameters(model)

# NOTE : `weight_decay` term denotes L2 regularization loss term
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.000)
initial_optimizer_state_dict = optimizer.state_dict()


LeNet(
  (fc1): MaskedLinear(in_features=784, out_features=512, bias=True)
  (fc2): MaskedLinear(in_features=512, out_features=512, bias=True)
  (fc3): MaskedLinear(in_features=512, out_features=512, bias=True)
  (fc4): MaskedLinear(in_features=512, out_features=10, bias=True)
)
Param name           Shape                          Type           
----------------------------------------------------------------------
fc1.weight           torch.Size([512, 784])         torch.float32  
fc1.mask             torch.Size([512, 784])         torch.float32  
fc1.bias             torch.Size([512])              torch.float32  
fc2.weight           torch.Size([512, 512])         torch.float32  
fc2.mask             torch.Size([512, 512])         torch.float32  
fc2.bias             torch.Size([512])              torch.float32  
fc3.weight           torch.Size([512, 512])         torch.float32  
fc3.mask             torch.Size([512, 512])         torch.float32  
fc3.bias             torch.Size([512]

In [0]:
def train(epochs, model):
    model.train()
    for epoch in range(epochs):

        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()

            # zero-out all the gradients corresponding to the pruned connections
            for name, p in model.named_parameters():
                if 'mask' in name:
                    continue
                tensor = p.data.cpu().numpy()
                grad_tensor = p.grad.data.cpu().numpy()
                grad_tensor = np.where(tensor==0, 0, grad_tensor)
                p.grad.data = torch.from_numpy(grad_tensor).to(device)

            optimizer.step()
            if batch_idx % log_interval == 0:
                done = batch_idx * len(data)
                percentage = 100. * batch_idx / len(train_loader)
                print(f'Train Epoch: {epoch} [{done:5}/{len(train_loader.dataset)} ({percentage:3.0f}%)]  Loss: {loss.item():.6f}')


def test(loader, model):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).sum().item()

        test_loss /= len(loader.dataset)
        accuracy = 100. * correct / len(loader.dataset)
        print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(loader.dataset)} ({accuracy:.2f}%)')
    return accuracy


In [9]:
# Initial training
print("--- Initial training ---")
train(epochs, model)
test_accuracy = test(test_loader, model)
train_accuracy = test(train_loader, model)
print("Test Accuracy:", test_accuracy)
print("Train Accuracy:", train_accuracy)


print("--- Before pruning ---")
print_nonzeros(model)

output_train, output_test, train_label, test_label = classifier_performance(model, train_loader, test_loader)
inference_accuracy=inference_via_confidence(output_train, output_test, train_label, test_label)
print("Maximum Accuracy:",inference_accuracy)



--- Initial training ---
Train Epoch: 0 [    0/60000 (  0%)]  Loss: 2.299644
Train Epoch: 0 [ 6400/60000 ( 11%)]  Loss: 0.549537
Train Epoch: 0 [12800/60000 ( 21%)]  Loss: 0.783319
Train Epoch: 0 [19200/60000 ( 32%)]  Loss: 0.372586
Train Epoch: 0 [25600/60000 ( 43%)]  Loss: 0.326012
Train Epoch: 0 [32000/60000 ( 53%)]  Loss: 0.502676
Train Epoch: 0 [38400/60000 ( 64%)]  Loss: 0.393806
Train Epoch: 0 [44800/60000 ( 75%)]  Loss: 0.448179
Train Epoch: 0 [51200/60000 ( 85%)]  Loss: 0.342985
Train Epoch: 0 [57600/60000 ( 96%)]  Loss: 0.298166
Train Epoch: 1 [    0/60000 (  0%)]  Loss: 0.435255
Train Epoch: 1 [ 6400/60000 ( 11%)]  Loss: 0.366023
Train Epoch: 1 [12800/60000 ( 21%)]  Loss: 0.416604
Train Epoch: 1 [19200/60000 ( 32%)]  Loss: 0.384105
Train Epoch: 1 [25600/60000 ( 43%)]  Loss: 0.307387
Train Epoch: 1 [32000/60000 ( 53%)]  Loss: 0.355735
Train Epoch: 1 [38400/60000 ( 64%)]  Loss: 0.553348
Train Epoch: 1 [44800/60000 ( 75%)]  Loss: 0.291713
Train Epoch: 1 [51200/60000 ( 85%)]  Lo

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')
model_save_name = 'fmnist.pt'
colab='Colab Notebooks'
path = F"/content/gdrive/My Drive/{colab}/{model_save_name}" 
torch.save(model.state_dict(), path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


#Pruning

In [14]:
model_save_name = 'fmnist.pt'
colab='Colab Notebooks'
path = F"/content/gdrive/My Drive/{colab}/{model_save_name}" 
temp = LeNet(mask=True).to(device)
temp.load_state_dict(torch.load(path))
test_accuracy = test(test_loader, temp)
train_accuracy = test(train_loader, temp)
print("Test Accuracy:", test_accuracy)
print("Train Accuracy:", train_accuracy)

Test set: Average loss: 0.7681, Accuracy: 8946/10000 (89.46%)
Test set: Average loss: 0.0788, Accuracy: 58470/60000 (97.45%)
Test Accuracy: 89.46
Train Accuracy: 97.45


In [15]:
print("--- Starting pruning ---")
temp.prune_by_std(1)
test_accuracy = test(test_loader,temp)
train_accuracy = test(train_loader, temp)
print("Test Accuracy:", test_accuracy)
print("Train Accuracy:", train_accuracy)
print("--- After pruning ---")
print_nonzeros(temp)

output_train, output_test, train_label, test_label = classifier_performance(temp, train_loader, test_loader)
inference_accuracy=inference_via_confidence(output_train, output_test, train_label, test_label)
print("Maximum Accuracy:",inference_accuracy)

--- Starting pruning ---
Pruning with threshold : 0.17446616291999817 for layer fc1
Pruning with threshold : 0.14549191296100616 for layer fc2
Pruning with threshold : 0.14017514884471893 for layer fc3
Test set: Average loss: 0.8931, Accuracy: 8676/10000 (86.76%)
Test set: Average loss: 0.2848, Accuracy: 55323/60000 (92.20%)
Test Accuracy: 86.76
Train Accuracy: 92.205
--- After pruning ---
fc1.weight           | nonzeros =  105173 /  401408 ( 26.20%) | total_pruned =  296235 | shape = (512, 784)
fc1.bias             | nonzeros =     512 /     512 (100.00%) | total_pruned =       0 | shape = (512,)
fc2.weight           | nonzeros =   68612 /  262144 ( 26.17%) | total_pruned =  193532 | shape = (512, 512)
fc2.bias             | nonzeros =     512 /     512 (100.00%) | total_pruned =       0 | shape = (512,)
fc3.weight           | nonzeros =   73603 /  262144 ( 28.08%) | total_pruned =  188541 | shape = (512, 512)
fc3.bias             | nonzeros =     512 /     512 (100.00%) | total_prune

#Retraining

In [0]:
print("--- Retraining ---")
optimizer = optim.Adam(temp.parameters(), lr=0.001)
train(epochs, temp)


print("--- After Retraining ---")
print_nonzeros(temp)

output_train, output_test, train_label, test_label = classifier_performance(temp, train_loader, test_loader)
inference_accuracy=inference_via_confidence(output_train, output_test, train_label, test_label)
print("Maximum Accuracy:",inference_accuracy)

--- Retraining ---
Train Epoch: 0 [    0/60000 (  0%)]  Loss: 0.161330
Train Epoch: 0 [ 6400/60000 ( 11%)]  Loss: 0.150198
Train Epoch: 0 [12800/60000 ( 21%)]  Loss: 0.070585
Train Epoch: 0 [19200/60000 ( 32%)]  Loss: 0.031430
Train Epoch: 0 [25600/60000 ( 43%)]  Loss: 0.067552
Train Epoch: 0 [32000/60000 ( 53%)]  Loss: 0.010059
Train Epoch: 0 [38400/60000 ( 64%)]  Loss: 0.043001
Train Epoch: 0 [44800/60000 ( 75%)]  Loss: 0.103635
Train Epoch: 0 [51200/60000 ( 85%)]  Loss: 0.141381
Train Epoch: 0 [57600/60000 ( 96%)]  Loss: 0.019617
Train Epoch: 1 [    0/60000 (  0%)]  Loss: 0.051503
Train Epoch: 1 [ 6400/60000 ( 11%)]  Loss: 0.045587
Train Epoch: 1 [12800/60000 ( 21%)]  Loss: 0.054236
Train Epoch: 1 [19200/60000 ( 32%)]  Loss: 0.019550
Train Epoch: 1 [25600/60000 ( 43%)]  Loss: 0.041788
Train Epoch: 1 [32000/60000 ( 53%)]  Loss: 0.046087
Train Epoch: 1 [38400/60000 ( 64%)]  Loss: 0.043991
Train Epoch: 1 [44800/60000 ( 75%)]  Loss: 0.046368
Train Epoch: 1 [51200/60000 ( 85%)]  Loss: 0.